In [1]:
import os
import glob
import datetime

# keras imports
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from keras.models import Model

# other imports
from sklearn.preprocessing import LabelEncoder
import numpy as np
import h5py
import json
import time

In [2]:
config={
  "model"           : "vgg16",
  "weights"         : "imagenet",
  "include_top"     : False,

  "train_path"      : "train",
  "test_path"       : "test",
  "features_path"   : "car_damage_check/features.h5",
  "labels_path"     : "car_damage_check/labels.h5",
  "results"         : "car_damage_check/results.txt",
  "classifier_path" : "car_damage_check/classifier.pickle",
  "model_path"      : "car_damage_check/model",

  "test_size"       : 0.20,
  "seed"            : 9,
  "num_classes"     : 2
}

In [3]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

In [4]:
model_name    = config["model"]
weights     = config["weights"]
include_top   = config["include_top"]
train_path    = config["train_path"]
test_path    = config["test_path"]
features_path   = config["features_path"]
labels_path   = config["labels_path"]
test_size     = config["test_size"]
results     = config["results"]
model_path    = config["model_path"]

In [5]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [6]:
if model_name == "vgg16":
    base_model = VGG16(weights=weights)
    model = Model(base_model.input,base_model.get_layer('fc1').output)
    image_size = (224, 224)
else:
    base_model = None

print ("Successfully loaded base model and model...")

Successfully loaded base model and model...


In [19]:
train_labels = os.listdir(train_path)

# encode the labels
print ("Encoding labels...")
le = LabelEncoder()
le.fit([tl for tl in train_labels])

# variables to hold features and labels
features = []
labels   = []

# loop over all the labels in the folder
count = 1
for i, label in enumerate(train_labels):
    cur_path = train_path + "/" + label
    count = 1
    for image_path in glob.glob(cur_path):
        img = image.load_img(image_path, target_size=image_size)
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        feature = model.predict(x)
        flat = feature.flatten()
        features.append(flat)
        labels.append(label)
        print ("Processed - " + str(count))
        count += 1
    print ("Completed label - " + label)

# encode the labels using LabelEncoder
le = LabelEncoder()
#le_labels = le.fit_transform(labels)

le_labels = np.array([1,1,1,0,0,0,0,0,0])
print(le_labels)
# get the shape of training labels
print ("Training labels: {}".format(le_labels))
print ("Training labels shape: {}".format(le_labels.shape))

Encoding labels...
Processed - 1
Completed label - 101.jpg
Processed - 1
Completed label - 102.jpg
Processed - 1
Completed label - 11.jpg
Processed - 1
Completed label - 43.jpg
Processed - 1
Completed label - 41.jpg
Processed - 1
Completed label - 40.jpg
Processed - 1
Completed label - 44.jpg
Processed - 1
Completed label - 47.jpg
Processed - 1
Completed label - 46.jpg
[1 1 1 0 0 0 0 0 0]
Training labels: [1 1 1 0 0 0 0 0 0]
Training labels shape: (9,)


In [20]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [21]:
h5f_data = h5py.File(features_path, 'w')
h5f_data.create_dataset('dataset_1', data=np.array(features))

h5f_label = h5py.File(labels_path, 'w')
h5f_label.create_dataset('dataset_1', data=np.array(le_labels))

h5f_data.close()
h5f_label.close()

# save model and weights
model_json = model.to_json()
with open(model_path + str(test_size) + ".json", "w") as json_file:
    json_file.write(model_json)

# save weights
model.save_weights(model_path + str(test_size) + ".h5")
print("Saved model and weights to disk..")

print ("Features and labels saved..")

# end time
end = time.time()
print ("End time - {}".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M")))

Saved model and weights to disk..
Features and labels saved..
End time - 2021-06-16 02:03


In [22]:
config={
  "model"           : "vgg16",
  "weights"         : "imagenet",
  "features_path"   : "car_damage_check/features.h5",
  "labels_path"     : "car_damage_check/labels.h5",
  "classifier_path" : "car_damage_check/classifier.pickle",
  "model_path"      : "cdd/car_damage_check/model",

  "test_size"       : 0.20,
  "seed"            : 9,
}

In [23]:
test_size  = config["test_size"]
seed = config["seed"]
features_path = config["features_path"]
labels_path = config["labels_path"]
classifier_path = config["classifier_path"]

In [24]:
# import features and labels
h5f_data  = h5py.File(features_path, 'r')
h5f_label = h5py.File(labels_path, 'r')

features_string = h5f_data['dataset_1']
labels_string   = h5f_label['dataset_1']

features = np.array(features_string)
labels   = np.array(labels_string)

h5f_data.close()
h5f_label.close()

In [25]:
features.shape

(9, 4096)

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

# verify the shape of features and labels
print(features.shape)
print(labels.shape)

print ("[INFO] training started...")
# split the training and testing data
(trainData, testData, trainLabels, testLabels) = train_test_split(np.array(features),
                                                                  np.array(labels),
                                                                  test_size=test_size,
                                                                  random_state=seed)

print ("[INFO] splitted train and test data...")
print ("[INFO] train data  : {}".format(trainData.shape))
print ("[INFO] test data   : {}".format(testData.shape))
print ("[INFO] train labels: {}".format(trainLabels.shape))
print ("[INFO] test labels : {}".format(testLabels.shape))

(9, 4096)
(9,)
[INFO] training started...
[INFO] splitted train and test data...
[INFO] train data  : (7, 4096)
[INFO] test data   : (2, 4096)
[INFO] train labels: (7,)
[INFO] test labels : (2,)


In [27]:
labels

array([1, 1, 1, 0, 0, 0, 0, 0, 0])

In [28]:
logmodel = LogisticRegression(max_iter=500)
logmodel.fit(trainData, trainLabels)

LogisticRegression(max_iter=500)

In [29]:
preds = logmodel.predict(testData)
print(preds)

[0 0]


In [33]:
import pickle
print ("[INFO] saving model...")
pickle.dump(logmodel, open(classifier_path, 'wb'))

[INFO] saving model...


In [34]:
import pickle
print ("[INFO] Loading the classifier...")
classifier = pickle.load(open(config["classifier_path"], 'rb'))

[INFO] Loading the classifier...


In [69]:
test_labels = os.listdir(test_path)
# variables to hold features and labels
features_new = []

# loop over all the labels in the folder
count = 1
for i, label in enumerate(test_labels):
    cur_path = test_path + "/" + label
    count = 1
    for image_path in glob.glob(cur_path):
        img = image.load_img(image_path, target_size=image_size)
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        feature = model.predict(x)
        flat = feature.flatten()
        features_new.append(flat)
        print ("Processed - " + str(count))
        count += 1
    print ("Completed label - " + label)


Processed - 1
Completed label - 99.jpg
Processed - 1
Completed label - 98.jpg
Processed - 1
Completed label - 12.jpg
Processed - 1
Completed label - 11.jpg
Processed - 1
Completed label - 34.jpg
Processed - 1
Completed label - 31.jpg


In [70]:
preds = logmodel.predict(features_new)
label_check1 = ['Car is Damaged', 'Car is not Damaged']

# Check if the car is damaged or not.
label_check1[preds[0]]

'Car is Damaged'

In [71]:
label_check1[preds[1]]

'Car is not Damaged'

In [72]:
label_check1[preds[2]]

'Car is not Damaged'

In [73]:
label_check1[preds[3]]

'Car is Damaged'

In [74]:
label_check1[preds[4]]

'Car is Damaged'

In [75]:
label_check1[preds[5]]

'Car is Damaged'